In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import csv
import os
import itertools

In [2]:
from scripts.utility import *
from scripts.features import *

# Features
It is a good idea to save our features in text files, so that we only have to calculate them once. Later on when we employ clustering, we can just read the values off of these files instead of iterating over the graphs again, which is time consuming.

## 1. Geometrical

### 1.1 Number of nodes and edges

In [ ]:
# Calculate the number of nodes and edges for each network and save them to files
with open('features/n_nodes.txt', 'w') as node_file, open('features/n_edges.txt', 'w') as edge_file:
    for network_id, G in graph_generator():
        node_file.write(network_id + '\t' + str(nx.number_of_nodes(G)) + '\n') # nx.number_of_nodes() function
        edge_file.write(network_id + '\t' + str(nx.number_of_edges(G)) + '\n') # use tab character \t, not two spaces
        
###NOTE: Have to pull features from github before running script
##IDEA: Can calculate every feature on the same loop

### 1.2 Average node degree

In [ ]:
# Calculate the average node degree, i.e. the average number of edges that are connected to a node,
# for each network and write them to a file
with open('features/average_node_degree.txt', 'w') as file:
    for network_id, G in graph_generator(): # check this access to graph_generator
        average_node_degree = np.mean(list(G.degree().values()))
        file.write(network_id + '\t' + str(average_node_degree) + '\n')

### 1.3 Vein density

Vein density **$\sigma$** is calculated by summing the total length of all veins and dividing by total leaf area.


In [ ]:
save_feature(vein_density, skip_existing=True, clean=False)

### 1.4 Areole area

Areole area **$A$** is obtained calculating basic cycle areas using the standard formula for the area of a general polygon.

In [ ]:
save_feature(areole_area, skip_existing=True, clean=False)

### 1.5 Areole density 

Areole density **$\rho_A$** is the total number of areoles divided by leaf area.

In [ ]:
save_feature(areole_density, skip_existing=True, clean=False)

### 1.6 Weighted vein thickness

Weighted vein thickness **$d$** is calculated as the total sum of the product radius*length of each
    individual vein segment divided by total vein length

In [ ]:
save_feature(weighted_vein_thickness, skip_existing=True, clean=False)

## 2. Topological

### 2.1 Nesting number

In [ ]:
save_feature(nesting_numbers, skip_existing=True, clean=True)

# Data Analysis

In [ ]:
generator = graph_generator()

In [ ]:
_, G = next(generator)

## Principal Component Analysis

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
X = np.array([[-1, -1, 2, 4], [-2, -1, 5, 6], [-3, -2, 1, -5], [1, 1, -3, -5], [2, 1, 6, 1], [3, 2, 2, 1]])
print('Data')
print(X)
#ipca = IncrementalPCA(n_components=2, batch_size=3)
pca = PCA(n_components=2)

pca.fit(X)

print('Data in PCA basis')
print(pca.transform(X))

In [ ]:
#### print('PCA Vectors')
print(pca.components_)

In [ ]:
data_reduced = np.dot(X - pca.mean_, pca.components_.T)
print(np.dot(pca.transform(X), pca.components_) + pca.mean_)
#print(np.dot(data_reduced, pca.components_))

In [ ]:
print(pca.inverse_transform(pca.transform(X)))

In [ ]:
topological_length = read_feature('topological_length')

In [ ]:
print(topological_length)

In [ ]:
nesting_numbers = read_feature('nesting_numbers')
unweighted_no_ext = nesting_numbers['BronxA_001'][3]

In [ ]:
print(nesting_numbers)

In [3]:
data = read_features()

In [9]:
network_ids = sorted(data['nesting_number_weighted'].keys())